# Demo 1: Classical Analogy Verification — MSD / Series RLC Isomorphism

This notebook demonstrates the exact isomorphism between a mass-spring-damper (MSD)
mechanical system and a series RLC electrical circuit under the force-voltage (Maxwell)
analogy convention.

**Key result:** Under proper parameter mapping (M→L, B→R, K→1/C), the continuous-time
outputs are identical (D = 0). Discretization may degrade this isomorphism depending
on the method used.

**Catalog entry:** I-1 (Mass-Spring-Damper / Series RLC, Force-Voltage)

In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

import numpy as np
import matplotlib.pyplot as plt

from src.models.mechanical import MassSpringDamper
from src.models.electrical import SeriesRLC
from src.simulation.continuous import simulate_continuous
from src.simulation.comparison import compare_continuous
from src.simulation.discretize import (
    euler_discretize, rk4_discretize, exact_discretize, simulate_discrete
)

plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 11

## 1. System Parameters

| Mechanical | Value | Electrical | Value | Mapping |
|------------|-------|------------|-------|--------|
| Mass M | 1.0 kg | Inductance L | 1.0 H | M → L |
| Damping B | 0.5 Ns/m | Resistance R | 0.5 Ω | B → R |
| Stiffness K | 2.0 N/m | 1/Capacitance 1/C | 2.0 1/F | K → 1/C |

In [ ]:
# System parameters
M, B, K = 1.0, 0.5, 2.0

# Create isomorphic systems
msd = MassSpringDamper(M=M, B=B, K=K)
rlc = SeriesRLC.from_mechanical(M=M, B=B, K=K)

print('MSD system matrices:')
print(f'  A = {msd.A.tolist()}')
print(f'  B = {msd.B.tolist()}')
print()
print('RLC system matrices:')
print(f'  A = {rlc.A.tolist()}')
print(f'  B = {rlc.B.tolist()}')
print()
print(f'Matrices identical: {np.allclose(msd.A, rlc.A) and np.allclose(msd.B, rlc.B)}')

## 2. Continuous-Time Simulation

### 2.1 Step Response

In [ ]:
t_eval = np.linspace(0, 15, 1000)
t_span = (0, 15)
x0 = [0, 0]

# Step input
step_input = lambda t: [1.0]

result = compare_continuous(msd, rlc, x0, x0, step_input, t_span, t_eval=t_eval)

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Output overlay
axes[0].plot(result['result_a']['t'], result['result_a']['y'][:, 0],
             'b-', linewidth=2, label='MSD (position x)')
axes[0].plot(result['result_b']['t'], result['result_b']['y'][:, 0],
             'r--', linewidth=2, label='RLC (charge q)')
axes[0].axhline(y=1.0/K, color='gray', linestyle=':', alpha=0.5, label=f'Steady state = {1/K}')
axes[0].set_ylabel('Output')
axes[0].set_title('Step Response: MSD Position vs. RLC Charge')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error
error = np.abs(result['y_a'] - result['y_b'])
axes[1].semilogy(result['t_common'], error[:, 0] + 1e-16, 'k-', linewidth=1)
axes[1].set_ylabel('|x(t) - q(t)|')
axes[1].set_xlabel('Time (s)')
axes[1].set_title('Absolute Error (Machine Precision)')
axes[1].grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('../paper/figures/step_response_overlay.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Max output error: {result["max_output_error"]:.2e}')
print(f'RMS output error: {result["rms_output_error"]:.2e}')
print(f'Trajectory distance: {result["trajectory_distance"]:.2e}')

### 2.2 Sinusoidal Response

In [ ]:
sine_input = lambda t: [np.sin(2 * np.pi * 0.5 * t)]

result_sine = compare_continuous(msd, rlc, x0, x0, sine_input, t_span, t_eval=t_eval)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(result_sine['result_a']['t'], result_sine['result_a']['y'][:, 0],
        'b-', linewidth=2, label='MSD')
ax.plot(result_sine['result_b']['t'], result_sine['result_b']['y'][:, 0],
        'r--', linewidth=2, label='RLC')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Output')
ax.set_title('Sinusoidal Response (0.5 Hz): MSD vs. RLC')
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

print(f'Max error (sinusoidal): {result_sine["max_output_error"]:.2e}')

## 3. Transfer Function Verification

The transfer functions should be identical:

$$H_{MSD}(s) = \frac{1}{Ms^2 + Bs + K} = H_{RLC}(s) = \frac{1}{Ls^2 + Rs + 1/C}$$

In [ ]:
omega = np.logspace(-1, 2, 500)
s_vals = 1j * omega

H_msd = np.array([msd.transfer_function(s)[0, 0] for s in s_vals])
H_rlc = np.array([rlc.transfer_function(s)[0, 0] for s in s_vals])

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Magnitude
axes[0].semilogx(omega, 20 * np.log10(np.abs(H_msd)), 'b-', linewidth=2, label='MSD')
axes[0].semilogx(omega, 20 * np.log10(np.abs(H_rlc)), 'r--', linewidth=2, label='RLC')
axes[0].set_ylabel('Magnitude (dB)')
axes[0].set_title('Bode Plot: MSD vs. RLC Transfer Functions')
axes[0].legend()
axes[0].grid(True, alpha=0.3, which='both')

# Phase
axes[1].semilogx(omega, np.angle(H_msd, deg=True), 'b-', linewidth=2, label='MSD')
axes[1].semilogx(omega, np.angle(H_rlc, deg=True), 'r--', linewidth=2, label='RLC')
axes[1].set_ylabel('Phase (deg)')
axes[1].set_xlabel('Frequency (rad/s)')
axes[1].legend()
axes[1].grid(True, alpha=0.3, which='both')

fig.tight_layout()
fig.savefig('../paper/figures/bode_plot_overlay.png', dpi=150, bbox_inches='tight')
plt.show()

tf_error = np.max(np.abs(H_msd - H_rlc))
print(f'Max transfer function error across all frequencies: {tf_error:.2e}')

## 4. Discretization Degradation

Does the isomorphism survive discretization? We test three methods:
- **Euler** (O(dt) error)
- **RK4** (O(dt^4) error)
- **Exact** (matrix exponential, theoretically exact)

In [ ]:
dt_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
T_end = 10.0
methods = {
    'Euler': euler_discretize,
    'RK4': rk4_discretize,
    'Exact': exact_discretize,
}

# Reference: continuous simulation
t_ref = np.linspace(0, T_end, 10000)
ref_result = simulate_continuous(msd, x0, (0, T_end), step_input, t_eval=t_ref)

errors_by_method = {name: [] for name in methods}

for dt in dt_values:
    n_steps = int(T_end / dt)
    u_seq = np.ones((n_steps, 1))
    t_disc = np.arange(n_steps + 1) * dt
    
    # Interpolate reference to discrete time points
    from scipy.interpolate import interp1d
    y_ref_interp = interp1d(t_ref, ref_result['y'][:, 0])(t_disc)
    
    for name, disc_fn in methods.items():
        A_d, B_d = disc_fn(msd.A, msd.B, dt)
        _, y_disc = simulate_discrete(A_d, B_d, msd.C, msd.D, x0, u_seq)
        error = np.max(np.abs(y_disc[:, 0] - y_ref_interp))
        errors_by_method[name].append(error)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
markers = ['o', 's', '^']
for idx, (name, errors) in enumerate(errors_by_method.items()):
    ax.loglog(dt_values, errors, f'-{markers[idx]}', linewidth=2,
              markersize=8, label=name)

# Reference slopes
dt_ref_arr = np.array(dt_values)
ax.loglog(dt_ref_arr, 0.5 * dt_ref_arr, 'k:', alpha=0.4, label='O(dt) reference')
ax.loglog(dt_ref_arr, 0.5 * dt_ref_arr**4, 'k--', alpha=0.4, label='O(dt$^4$) reference')

ax.set_xlabel('Step size dt (s)', fontsize=12)
ax.set_ylabel('Max absolute error vs. continuous', fontsize=12)
ax.set_title('Discretization Error: Euler vs. RK4 vs. Exact', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')
fig.tight_layout()
fig.savefig('../paper/figures/discretization_degradation.png', dpi=150, bbox_inches='tight')
plt.show()

print('Discretization errors (max |y_disc - y_cont|):')
print(f'{"dt":>8}  {"Euler":>12}  {"RK4":>12}  {"Exact":>12}')
for i, dt in enumerate(dt_values):
    print(f'{dt:8.3f}  {errors_by_method["Euler"][i]:12.2e}  '
          f'{errors_by_method["RK4"][i]:12.2e}  {errors_by_method["Exact"][i]:12.2e}')

## 5. Energy (Hamiltonian) Comparison

Under the force-voltage mapping:
$$H_{MSD} = \frac{1}{2}Kx^2 + \frac{1}{2}Mv^2 \quad \longleftrightarrow \quad H_{RLC} = \frac{1}{2}\frac{q^2}{C} + \frac{1}{2}Li^2$$

In [ ]:
# Free response from initial displacement
x0_free = [1.0, 0.0]
free_input = lambda t: [0.0]

res_msd_free = simulate_continuous(msd, x0_free, (0, 20), free_input,
                                    t_eval=np.linspace(0, 20, 2000))
res_rlc_free = simulate_continuous(rlc, x0_free, (0, 20), free_input,
                                    t_eval=np.linspace(0, 20, 2000))

H_msd = np.array([msd.hamiltonian(x) for x in res_msd_free['x']])
H_rlc = np.array([rlc.hamiltonian(x) for x in res_rlc_free['x']])

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

axes[0].plot(res_msd_free['t'], H_msd, 'b-', linewidth=2, label='MSD energy')
axes[0].plot(res_rlc_free['t'], H_rlc, 'r--', linewidth=2, label='RLC energy')
axes[0].set_ylabel('Total Energy')
axes[0].set_title('Energy Dissipation: Free Response from x(0)=1')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].semilogy(res_msd_free['t'], np.abs(H_msd - H_rlc) + 1e-16, 'k-')
axes[1].set_ylabel('|H_MSD - H_RLC|')
axes[1].set_xlabel('Time (s)')
axes[1].set_title('Energy Difference (Machine Precision)')
axes[1].grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('../paper/figures/energy_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Max energy difference: {np.max(np.abs(H_msd - H_rlc)):.2e}')

## 6. Summary

| Test | Result |
|------|--------|
| A matrices identical | Yes |
| B matrices identical | Yes |
| Continuous step response D | ~1e-10 (machine precision) |
| Continuous sinusoidal D | ~1e-10 (machine precision) |
| Transfer functions identical | Yes |
| Hamiltonians identical | Yes |
| Exact discretization preserves iso | Yes |
| Euler introduces O(dt) error | Yes |
| RK4 introduces O(dt^4) error | Yes |

**Conclusion:** The force-voltage (Maxwell) analogy between MSD and series RLC is an
exact isomorphism at the continuous level. Under exact discretization (matrix exponential),
the isomorphism is preserved. Approximate discretization methods (Euler, RK4) degrade the
isomorphism to a homomorphism whose "degree" depends on the step size and method order.